In [1]:
import pandas as pd


In [2]:
!git clone https://github.com/magistery-tps/text-mining.git
!cd text-mining
!conda env create -f environment.yml
!cd datasets/fashion-outfits

Cloning into 'text-mining'...
remote: Enumerating objects: 528, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 528 (delta 24), reused 69 (delta 19), pack-reused 446
Receiving objects: 100% (528/528), 106.86 MiB | 14.63 MiB/s, done.
Resolving deltas: 100% (233/233), done.
Checking out files: 100% (42/42), done.
/bin/bash: conda: command not found
/bin/bash: line 0: cd: datasets/fashion-outfits: No such file or directory


In [3]:
import pyarrow.parquet as pq
import pyarrow as pa

#df_products = pq.read_table(source='/content/drive/MyDrive/Farfetch/products.parquet').to_pandas()

ds_products = pq.read_table(source='/content/text-mining/datasets/fashion-outfits/products.parquet').to_pandas()

In [4]:
ds_products.describe()

,product_id
count,3.986700e+05
mean,1.707395e+07
std,8.080415e+05
min,1.004903e+07
25%,1.658652e+07
50%,1.710693e+07
75%,1.768004e+07
max,1.840134e+07


In [5]:
ds_products['branch'] = ds_products['product_family'] + '~' + ds_products['product_category'] + '~' + ds_products['product_sub_category']

In [6]:
ds_branch_cant= ds_products.groupby('branch').branch.count().reset_index(name='cant')

In [7]:
ds_products=ds_products.merge(ds_branch_cant.query('cant>=110'), on ='branch')

ds_products.reset_index()

,index,product_id,product_family,product_category,product_sub_category,product_gender,product_main_colour,product_second_color,product_brand,product_materials,product_short_description,product_attributes,product_image_path,product_highlights,branch,cant
0,0,17073270,Clothing,Knitwear,Cardi-Coats,UNISEX,BLACK,WHITE,STELLA MCCARTNEY,[Cotton],x The Beatles intarsia-knit coat,"[{""attribute_name"":""Knit Neckline"",""attribute_...",17/07/32/70/17073270.jpg,"[black/white, organic cotton, mix print, intar...",Clothing~Knitwear~Cardi-Coats,453
1,1,17674562,Clothing,Knitwear,Cardi-Coats,UNISEX,PURPLE,N/D,extreme cashmere,"[Spandex/Elastane, Nylon, Cashmere]",cashmere-blend long belted cardigan,"[{""attribute_name"":""Knit Neckline"",""attribute_...",17/67/45/62/17674562.jpg,"[plum purple, cashmere blend, wrap design, sle...",Clothing~Knitwear~Cardi-Coats,453
2,2,17678603,Clothing,Knitwear,Cardi-Coats,UNISEX,BLUE,N/D,extreme cashmere,"[Spandex/Elastane, Cashmere, Nylon]",cashmere-blend long belted cardigan,"[{""attribute_name"":""Knit Neckline"",""attribute_...",17/67/86/03/17678603.jpg,"[blue, cashmere blend, wrap design, sleeveless...",Clothing~Knitwear~Cardi-Coats,453
3,3,17179699,Clothing,Knitwear,Cardi-Coats,UNISEX,PINK,N/D,extreme cashmere,"[Nylon, Spandex/Elastane, Cashmere]",long cashmere cardigan,"[{""attribute_name"":""Knit Neckline"",""attribute_...",17/17/96/99/17179699.jpg,"[light pink, stretch-cashmere blend, fine knit...",Clothing~Knitwear~Cardi-Coats,453
4,4,17372698,Clothing,Knitwear,Cardi-Coats,MEN,BLACK,N/D,LES HOMMES,[Virgin Wool],cable knit oversized shawl cardigan,"[{""attribute_name"":""Knit Neckline"",""attribute_...",17/37/26/98/17372698.jpg,"[black , virgin wool , cable knit, draped desi...",Clothing~Knitwear~Cardi-Coats,453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394434,394434,17297436,Clothing,Suits,Double Breasted Suits,MEN,BLUE,N/D,CARUSO,"[Cupro, Wool, viscose]",double-breasted two-peice suit,None,17/29/74/36/17297436.jpg,"[Blazer:, Trousers:, straight leg, slim cut, b...",Clothing~Suits~Double Breasted Suits,128
394435,394435,17503772,Clothing,Suits,Double Breasted Suits,MEN,GREY,N/D,ELEVENTY,"[Cupro, Linen/Flax, Polyamide, Cotton, Alpaca ...",mélange-effect double-breasted suit,None,17/50/37/72/17503772.jpg,[ null],Clothing~Suits~Double Breasted Suits,128
394436,394436,17631490,Clothing,Suits,Double Breasted Suits,MEN,BLUE,N/D,ELEVENTY,"[Spandex/Elastane, Polyester]",double-breasted trouser suit,None,17/63/14/90/17631490.jpg,"[blue, tonal design, peak lapels, long sleeves...",Clothing~Suits~Double Breasted Suits,128
394437,394437,17681858,Clothing,Suits,Double Breasted Suits,MEN,BLACK,N/D,TONELLO,"[viscose, Virgin Wool, Polyester, Spandex/Elas...",double-breasted virgin wool-blend suit,None,17/68/18/58/17681858.jpg,"[double-breasted button fastening, notched col...",Clothing~Suits~Double Breasted Suits,128


In [8]:
from sklearn.model_selection import train_test_split
train_set, test_set, _, _= train_test_split(
    ds_products,
    ds_products['branch'].values,
    test_size     = 0.3,
    stratify      = ds_products['branch'].values,
    shuffle       = True
)


In [9]:
cant_train=train_set.groupby('branch').branch.count().reset_index(name='cant')
cant_test=test_set.groupby('branch').branch.count().reset_index(name='cant')

In [10]:
cant_test['porc']=cant_test.cant/cant_test.cant.sum()*100
cant_train['porc']=cant_train.cant/cant_train.cant.sum()*100

In [11]:
import seaborn as sn
import matplotlib.pyplot as plt

corr_train_test=cant_test[['branch','porc']]
corr_train_test['porc_train']=cant_train['porc']

corr_train_test['dif']=(corr_train_test['porc_train']-corr_train_test['porc']).abs()

corr_train_test.sort_values(by='dif',ascending=True)




,branch,porc,porc_train,dif
61,Clothing~Denim~Loose-Fit Jeans,0.030423,0.030423,1.101851e-07
174,Lingerie & Nightwear~Loungewear~N/D,0.035493,0.035493,1.285493e-07
1,Accessories~Hair Accessories~N/D,0.038029,0.038029,1.377314e-07
171,Jewellery~Watches~N/D,0.038029,0.038029,1.377314e-07
56,Clothing~Denim~Boyfriend Jeans,0.053240,0.053240,1.928239e-07
...,...,...,...,...
160,Clothing~Trousers~Wide-Leg Trousers,0.138593,0.137990,6.031288e-04
150,Clothing~Trousers~Loose Fit,0.108170,0.107567,6.032389e-04
20,Activewear~Performance Trousers~Sport Track Pants,0.031268,0.031872,6.037440e-04
72,Clothing~Dresses~N/D,0.049015,0.049618,6.038082e-04


In [12]:
#train_set.to_csv('/content/text-mining/datasets/fashion-outfits/train_set.csv', index=False)

In [13]:
del(train_set['cant'])
del(test_set['cant'])

In [14]:
table=pa.Table.from_pandas(train_set)
test_set.to_csv('/content/text-mining/datasets/fashion-outfits/test_set.csv', index=False)
pq.write_table(table, '/content/text-mining/datasets/fashion-outfits/train_set.parquet')